# **ETL pipeline**

- The goal here is to retrieve the raw data and proceed to feature engineering
- At the end of this pipeline we will have 2 tables : the instances (that contain our explanatory variables) and the labels (that contains the target variable)
- We split the 2 because in real life we have 2 phases : 
    - Training phase : we have the instances & labels -> feature engineering -> pre-processing (imputation, encoder fit + transform) -> model training
    - Inference phase : we don't have labels when using the model in production -> feature engineering -> pre-processing (imputation, fitted encoder transform) -> trained model prediction
- This pipeline contains the feature engineering part because the model training & inference should start directly with pre-processing
- The ID variable is the policy_number

In [1]:
import sys
from pathlib import Path
import logging 

LOGGER = logging.getLogger(__name__)
sys.path.append(str(Path("../src").resolve()))

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## **Parameters**

`unusable_columns_to_remove` : justifications on why we are removing these columns
- `insured_sex` : in France it is prohibited to use gender data in most cases as these are very sensitive personal data (GDPR)
- `insured_relationship` : personnal data related to gender 
- `incident_location` : 
    - too granular, we already have the state & city, 
    - could have been interesting to calculate the distance between incident location and the home adress
    - we would need the latitude/longitude of these 2 adresses 
- `auto_model` : 
    - too granular, the auto brand already tells the information about the type (premium, cheap,...) of the vehicle
    - it would have been interesting to have the car price 

In [2]:
unusable_columns_to_remove = [
    "insured_sex", 
    "insured_relationship", 
    "incident_location",  
    "auto_model",
    "_c39"
]

date_columns_list = ["policy_bind_date", "incident_date"]

## **Inputs loading**

In [3]:
import polars as pl 
from pathlib import Path

insurance_claims = pl.read_csv(source=(Path().cwd().parent / "data/01_raw/insurance_claims.csv").as_posix(), separator=",")

## **Execution**

In [4]:
import functions.etl.features_creation_functions as etl_features_creation

LOGGER.info("-------------------- Filtering unusable columns --------------------")
insurance_claims_without_unusable_columns = etl_features_creation.filter_unusable_columns(
    data = insurance_claims, 
    columns_to_remove=unusable_columns_to_remove
)
LOGGER.info("-------------------- Converting date as str columns to date columns --------------------")
insurance_claims_with_datetime_columns = etl_features_creation.convert_dates_str_to_datetime(
    data = insurance_claims_without_unusable_columns, 
    date_columns_list = date_columns_list
)
LOGGER.info("-------------------- Splitting the CSL column --------------------")
insurance_claims_with_split_limits = etl_features_creation.split_csl_column(data=insurance_claims_with_datetime_columns)
LOGGER.info("-------------------- Creating timedelta between incident_date and bind_date --------------------")
insurance_claims_with_incident_bind_date_timedelta = etl_features_creation.calculate_timedelta_between_incident_date_and_bind_date(data=insurance_claims_with_split_limits)
LOGGER.info("-------------------- Splitting instances & labels --------------------")
instances,labels = etl_features_creation.split_instances_labels(
    data=insurance_claims_with_incident_bind_date_timedelta, 
    target_col="fraud_reported", 
    id_col="policy_number"
)
LOGGER.info("Saving the final table...")
target_directory = (Path().cwd().parent / "data/02_intermediate/")
target_directory.mkdir(parents=True, exist_ok=True)
instances.write_parquet((target_directory / "instances.parquet").as_posix())
labels.write_parquet((target_directory / "labels.parquet").as_posix())

2025-07-11 00:11:40,341 : INFO : -------------------- Filtering unusable columns --------------------
2025-07-11 00:11:40,343 : INFO : data.shape = (1000, 40) : data['policy_number'].n_unique() = 1000
2025-07-11 00:11:40,345 : INFO : filtered_data.shape = (1000, 35) : filtered_data['policy_number'].n_unique() = 1000
2025-07-11 00:11:40,345 : INFO : -------------------- Converting date as str columns to date columns --------------------
2025-07-11 00:11:40,353 : INFO : -------------------- Splitting the CSL column --------------------
2025-07-11 00:11:40,356 : INFO : data.shape = (1000, 37) : data['policy_number'].n_unique() = 1000
2025-07-11 00:11:40,358 : INFO : -------------------- Creating timedelta between incident_date and bind_date --------------------
2025-07-11 00:11:40,361 : INFO : data.shape = (1000, 37) : data['policy_number'].n_unique() = 1000
2025-07-11 00:11:40,362 : INFO : -------------------- Splitting instances & labels --------------------
2025-07-11 00:11:40,364 : IN

In [5]:
instances.head(2)

months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_education_level,insured_occupation,insured_hobbies,capital-gains,capital-loss,incident_date,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_year,per_person_limit,per_accident_limit,nb_years_between_incident_and_bind_date
i64,i64,i64,date,str,i64,f64,i64,i64,str,str,str,i64,i64,date,str,str,str,str,str,str,i64,i64,str,i64,i64,str,i64,i64,i64,i64,str,i64,f64,f64,f64
328,48,521585,2014-10-17,"""OH""",1000,1406.91,0,466132,"""MD""","""craft-repair""","""sleeping""",53300,0,2015-01-25,"""Single Vehicle Collision""","""Side Collision""","""Major Damage""","""Police""","""SC""","""Columbus""",5,1,"""YES""",1,2,"""YES""",71610,6510,13020,52080,"""Saab""",2004,250.0,500.0,2.3655e7
228,42,342868,2006-06-27,"""IN""",2000,1197.22,5000000,468176,"""MD""","""machine-op-inspct""","""reading""",0,0,2015-01-21,"""Vehicle Theft""","""?""","""Minor Damage""","""Police""","""VA""","""Riverwood""",8,1,"""?""",0,0,"""?""",5070,780,780,3510,"""Mercedes""",2007,250.0,500.0,7.4040e8


In [6]:
labels.head(2)

policy_number,fraud_reported
i64,str
521585,"""Y"""
342868,"""Y"""
